In [1]:
import pandas as pd
import datetime
import pymongo
import json
from bson import Binary, Code, json_util, ObjectId
from bson.json_util import dumps, RELAXED_JSON_OPTIONS
import pygeoj
import os
import itertools 

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.education_data
# collection = db.grad_data_2013

In [3]:
# files
rootdir = './data/4YearGraduation'

# lists defined
districts_list = []
years = ['2018','2015','2014','2016','2017']

# for loop to get all files
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        data = os.path.join(subdir, file)
        districts_list.append(data)
#         districts_list.sort()
districts_list.pop(0)
# districts_list.reverse()
df=[]
#  add all 4 year graduation from 2013 to 2018 to mongod
for (file, year) in zip(districts_list, years):
    data = file
#     defined variables labels
    grad_label = "grad_"+ year
    df_label = grad_label +"_df"
    dic_label = "to_dic_" + year
    db_label = "grad_data_" + year
    collection = db[db_label]
    
#     Read data & add to dataframe
    grad_label = pd.read_csv(data)
    df_label = pd.DataFrame(grad_label)
#     replace * with 0
    df_label['FOUR_YR_GRAD_RATE'] = df_label['FOUR_YR_GRAD_RATE'].str.replace('*','0')
    df_label['FOUR_YR_ADJ_COHORT_COUNT'] = df_label['FOUR_YR_ADJ_COHORT_COUNT'].str.replace('*','0')
    df_label['GRADUATED_COUNT'] = df_label['GRADUATED_COUNT'].str.replace('*','0')
    df_label['FOUR_YR_GRAD_RATE'] = df_label['FOUR_YR_GRAD_RATE'].str.replace('-','0')
    df_label['FOUR_YR_ADJ_COHORT_COUNT'] = df_label['FOUR_YR_ADJ_COHORT_COUNT'].str.replace('-','0')
    df_label['GRADUATED_COUNT'] = df_label['GRADUATED_COUNT'].str.replace('-','0')
    # set previous change to numeric (int)
    df_label['FOUR_YR_GRAD_RATE'] = pd.to_numeric(df_label['FOUR_YR_GRAD_RATE'])
    df_label['FOUR_YR_ADJ_COHORT_COUNT'] = pd.to_numeric(df_label['FOUR_YR_ADJ_COHORT_COUNT'])
    df_label['GRADUATED_COUNT'] = pd.to_numeric(df_label['GRADUATED_COUNT'])
    sum_df = df_label.groupby(['COUNTY_NAME', 'SUBGROUP']).agg({'GRADUATED_COUNT': 'sum'}).reset_index()
    sum_to_dic = sum_df.to_dict('records')
    
    city_sum = sum_df = df_label.groupby(['DISTRICT_NAME', 'SUBGROUP','SCHOOL_NAME']).agg({'GRADUATED_COUNT': 'sum'}).reset_index()
    city_to_dic = sum_df.to_dict('records')
    
    #     add data to dictionary
    dic_label = df_label.to_dict('records')
    
#     drop collection if duplicate
    db.collection.drop()
#     add data to collection
    collection.insert_many(dic_label)
    
    db_county = 'county_grad_total' + year
    collection_county = db[db_county]
    
    db.collection_county.drop()
    collection_county.insert_many(sum_to_dic)
    
    db_district = 'district_grad_total' + year
    collection_district = db[db_district]
    
    db.collection_district.drop()
    collection_district.insert_many(city_to_dic)
    


In [4]:
# files
rootdir = './data/district'

# lists defined
budget_list = []
years = ['2020','2018-2019','2017-2018','2016-2017','2015-2016','2014-2015','2013-2014']

# for loop to get all files
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        data = os.path.join(subdir, file)
        budget_list.append(data)
        budget_list.sort()
        budget_list.reverse()
        

# df_budget=[]
#  add all 4 year graduation from 2013 to 2018 to mongod
for (file, year) in zip(budget_list, years):
    data = file
#     defined variables labels
    budget_label = "district_budget"+ year
    df_label = budget_label +"_df"
    dic_label = "to_dic_" + year
    db_label = "district_budget_" + year
    collection_budget = db[db_label]
    
    
#     Read data & add to dataframe
    grad_label = pd.read_csv(data)
    df_label = pd.DataFrame(grad_label)
#     add data to dictionary
    dic_label = df_label.to_dict('records')
#     drop collection if duplicate
    db.collection_budget.drop()
#     add data to collection
    collection_budget.insert_many(dic_label)
    
#     budget by county & insert to mongod
    sum_df = df_label.groupby(['COUNTY_NAME', 'YEAR']).agg({'TOTAL': 'sum'}).reset_index()
    sum_to_dic = sum_df.to_dict('records')
    
    db_counties = 'county_budget' + year
    collection_county = db[db_counties]
    
    db.collection_county.drop()
    collection_county.insert_many(sum_to_dic)

#     budget by cities & insert to mongod
    city_sum = df_label.groupby(['DISTRICT_NAME', 'YEAR']).agg({'TOTAL': 'sum'}).reset_index()
    city_to_dic = city_sum.to_dict('records')
    
    db_cities = 'city_budget' + year
    collection_cities = db[db_cities]
    
    db.collection_cities.drop()
    collection_cities.insert_many(city_to_dic)
